In [1]:
import json
import pandas as pd
import numpy as np

In [2]:
half = "2H"
max_min = 15
min_min = max_min - 15
max_sec = max_min*60
min_sec = min_min*60
table_name = "final_in_poss_df"

In [3]:
print(f'{table_name}_{half}_{min_min}_{max_min}.csv')

final_in_poss_df_2H_0_15.csv


In [4]:
with open("../../events/events_England.json", "r") as file:
    event_data_2 = json.load(file)

df2 = pd.DataFrame(event_data_2)
df2 = df2[df2.matchPeriod == half]
df2 = df2[df2.eventSec >= min_sec]
if max_min < 45:
    df2 = df2[df2.eventSec < max_sec]

with open("../../events/events_France.json", "r") as file:
    event_data_3 = json.load(file)

df3 = pd.DataFrame(event_data_3)
df3 = df3[df3.matchPeriod == half]
df3 = df3[df3.eventSec >= min_sec]
if max_min < 45:
    df3 = df3[df3.eventSec < max_sec]


with open("../../events/events_Germany.json", "r") as file:
    event_data_4 = json.load(file)

df4 = pd.DataFrame(event_data_4)
df4 = df4[df4.matchPeriod == half]
df4 = df4[df4.eventSec >= min_sec]
if max_min < 45:
    df4 = df4[df4.eventSec < max_sec]


with open("../../events/events_Italy.json", "r") as file:
    event_data_5 = json.load(file)

df5 = pd.DataFrame(event_data_5)
df5 = df5[df5.matchPeriod == half]
df5 = df5[df5.eventSec >= min_sec]
if max_min < 45:
    df5 = df5[df5.eventSec < max_sec]


with open("../../events/events_Spain.json", "r") as file:
    event_data_6 = json.load(file)

df6 = pd.DataFrame(event_data_6)
df6 = df6[df6.matchPeriod == half]
df6 = df6[df6.eventSec >= min_sec]
if max_min < 45:
    df6 = df6[df6.eventSec < max_sec]


with open("../../events/teams.json", "r") as file:
    teams_data = json.load(file)

teams_df = pd.DataFrame(teams_data)

In [5]:
all_events_df = pd.concat([df2, df3, df4, df5, df6], ignore_index=True)

# Feature engineering - Pass DF - Pass Motifs

In [6]:
passes_df = all_events_df[all_events_df['eventName'] == 'Pass']

# Concatenating teamId and matchId to create a unique identifier for each team in each match
passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)

<ipython-input-6-846069bd07cf>:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['uniqueTeamId'] = passes_df['teamId'].astype(str) + "---" + passes_df['matchId'].astype(str)


In [7]:
# Function to classify pass motifs
def classify_pass_motif(passes):
    players = passes['playerId'].values
    if len(set(players)) == 2:  # ABAB
        return 'ABAB'
    elif len(set(players)) == 3:
        if players[0] == players[2] or players[1] == players[3]:  # ABCA or ABCB
            return 'ABCA' if players[0] == players[2] else 'ABCB'
        else:
            return 'ABCD'  # ABCD for distinct players or other scenarios
    return 'ABCD'

# Apply the classification within a rolling window
def apply_motif_classification(df):
    # Ensure the DataFrame is sorted by event time
    df = df.sort_values(by='eventSec')
    motifs = []
    # Using a rolling window of 4 events
    for i in range(len(df) - 3):
        window = df.iloc[i:i+4]
        motif = classify_pass_motif(window)
        motifs.append((window.iloc[-1]['uniqueTeamId'], motif))
    return pd.DataFrame(motifs, columns=['uniqueTeamId', 'Motif'])

In [8]:
motif_classification_results = passes_df.groupby('uniqueTeamId').apply(apply_motif_classification)

In [9]:
motif_classification_results.reset_index(drop=True, inplace=True)

In [10]:
motif_classification_results

,uniqueTeamId,Motif
0,10531---2499726,ABCA
1,10531---2499726,ABCD
2,10531---2499726,ABCD
3,10531---2499726,ABCD
4,10531---2499726,ABCD
...,...,...
243626,756---2565923,ABCB
243627,756---2565923,ABCA
243628,756---2565923,ABCD
243629,756---2565923,ABCD


In [11]:
motif_counts = motif_classification_results.groupby(['uniqueTeamId', 'Motif']).size().unstack(fill_value=0)

In [12]:
motif_ratios = motif_counts.div(motif_counts.sum(axis=1), axis=0)
# df to be used in clustering

In [13]:
motif_ratios

Motif,ABAB,ABCA,ABCB,ABCD
uniqueTeamId,,,,
10531---2499726,0.031746,0.079365,0.063492,0.825397
10531---2499736,0.032258,0.129032,0.145161,0.693548
10531---2499741,0.041667,0.125000,0.083333,0.750000
10531---2499757,0.040000,0.106667,0.106667,0.746667
10531---2499766,0.076923,0.153846,0.192308,0.576923
...,...,...,...,...
756---2565880,0.106383,0.085106,0.085106,0.723404
756---2565892,0.041667,0.083333,0.069444,0.805556
756---2565904,0.000000,0.075472,0.075472,0.849057


In [14]:
motif_stats = motif_ratios.agg(['mean', 'std']).T

In [15]:
motif_stats

,mean,std
Motif,,
ABAB,0.035836,0.031052
ABCA,0.106361,0.038381
ABCB,0.107508,0.038706
ABCD,0.750295,0.083721


# Feature engineering - Shot DF

In [16]:
shot_df = all_events_df[all_events_df['eventName'] == 'Shot']
shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)

<ipython-input-16-396a5f5d4462>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['uniqueTeamId'] = shot_df['teamId'].astype(str) + "---" + shot_df['matchId'].astype(str)


In [17]:
import math

def calculate_shot_distance(row):
    if row['positions'] is not None and len(row['positions']) > 0:
        loc = row['positions'][0]
        # Normalize locations
        x = loc['x'] * (120 / 100.0)  # Normalizing the x coordinate
        y = loc['y'] * (80 / 100.0)   # Normalizing the y coordinate
        # Calculate Euclidean distance from (0,0)
        distance = math.sqrt((120-x)**2 + (80-y)**2)
        return distance
    return None

# Apply the function to calculate shot distance and add it to the DataFrame
shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)

<ipython-input-17-103ed8455f10>:15: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_distance'] = shot_df.apply(calculate_shot_distance, axis=1)


In [18]:
def classify_shot_by_distance(distance):
    if distance <= 38:
        return "Near"
    elif distance <= 52:
        return "Middle Distance"
    else:
        return "Far"

# Assuming shot_df has a 'shot_distance' column calculated as before
shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)

# Grouping shots by their class and counting
shot_class_counts = shot_df.groupby('uniqueTeamId')['shot_class'].value_counts().unstack(fill_value=0)
shot_counts = shot_df.groupby('uniqueTeamId')['subEventName'].value_counts().unstack(fill_value=0)

<ipython-input-18-0451b5045683>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  shot_df['shot_class'] = shot_df['shot_distance'].apply(classify_shot_by_distance)


In [19]:
shot_df_final = pd.concat([shot_counts, shot_class_counts], axis=1)

In [20]:
shot_df_final

,Shot,Far,Middle Distance,Near
uniqueTeamId,,,,
10531---2499726,3,1,0,2
10531---2499736,1,1,0,0
10531---2499741,1,0,0,1
10531---2499757,2,0,1,1
10531---2499777,3,0,1,2
...,...,...,...,...
756---2565854,2,0,1,1
756---2565880,2,0,1,1
756---2565892,2,1,1,0


In [21]:
shot_df_final.rename(columns = {'Shot':'Total Shots'}, inplace = True) 

shot_df_final["Far"] = shot_df_final["Far"]/shot_df_final["Total Shots"]
shot_df_final["Middle Distance"] = shot_df_final["Middle Distance"]/shot_df_final["Total Shots"]
shot_df_final["Near"] = shot_df_final["Near"]/shot_df_final["Total Shots"]


shot_df_final.rename(columns = {'Near':'Near Shots Ratio', 'Middle Distance':'Middle Distance Shots Ratio', 'Far':'Far Shots Ratio',}, inplace = True) 

In [22]:
shot_df_final

,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio
uniqueTeamId,,,,
10531---2499726,3,0.333333,0.000000,0.666667
10531---2499736,1,1.000000,0.000000,0.000000
10531---2499741,1,0.000000,0.000000,1.000000
10531---2499757,2,0.000000,0.500000,0.500000
10531---2499777,3,0.000000,0.333333,0.666667
...,...,...,...,...
756---2565854,2,0.000000,0.500000,0.500000
756---2565880,2,0.000000,0.500000,0.500000
756---2565892,2,0.500000,0.500000,0.000000


# Feature engineering - Pass DF - Pass Types

In [23]:
def determine_zone(position):
    if 'x' in position and 'y' in position:
        x, y = position['x'], position['y']
        zone = int(x // 33.333) + 1
        return zone
    else:
        return None
    
# Apply the function to determine zones for each event
passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)

<ipython-input-23-1dcee42e8a09>:10: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['Zone'] = passes_df['positions'].apply(lambda pos: determine_zone(pos[0]) if len(pos) > 0 else None)


In [24]:
passes_df.loc[passes_df['Zone'] == 4, ['Zone']] = 3

In [25]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone
0,8,Simple pass,[{'id': 1801}],14763,"[{'y': 50, 'x': 50}, {'y': 42, 'x': 38}]",2499719,Pass,1631,2H,2.836169,85,177960135,1631---2499719,2
1,8,High pass,[{'id': 1802}],8653,"[{'y': 42, 'x': 38}, {'y': 16, 'x': 74}]",2499719,Pass,1631,2H,5.311682,83,177960136,1631---2499719,2
2,8,Head pass,"[{'id': 1401}, {'id': 1801}]",167145,"[{'y': 84, 'x': 26}, {'y': 68, 'x': 32}]",2499719,Pass,1609,2H,8.449574,82,177960133,1609---2499719,1
3,8,Simple pass,[{'id': 1802}],49876,"[{'y': 68, 'x': 32}, {'y': 100, 'x': 27}]",2499719,Pass,1609,2H,10.458703,85,177960134,1609---2499719,1
12,8,High pass,[{'id': 1802}],370224,"[{'y': 76, 'x': 8}, {'y': 75, 'x': 44}]",2499719,Pass,1609,2H,77.088925,83,177960140,1609---2499719,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498274,8,Simple pass,[{'id': 1801}],286994,"[{'y': 9, 'x': 28}, {'y': 16, 'x': 32}]",2565927,Pass,682,2H,886.983156,85,253301948,682---2565927,1
498276,8,Simple pass,[{'id': 1801}],20623,"[{'y': 41, 'x': 47}, {'y': 82, 'x': 60}]",2565927,Pass,682,2H,891.850784,85,253301950,682---2565927,2
498277,8,Simple pass,[{'id': 1801}],3529,"[{'y': 82, 'x': 60}, {'y': 66, 'x': 48}]",2565927,Pass,682,2H,895.998317,85,253301951,682---2565927,2
498278,8,Simple pass,[{'id': 1801}],364860,"[{'y': 66, 'x': 48}, {'y': 46, 'x': 52}]",2565927,Pass,682,2H,898.157657,85,253301953,682---2565927,2


In [26]:
import math

def calculate_pass_details_from_single_column(row):
    if row['positions'] is not None and len(row['positions']) == 2:
        # Extract and normalize start and end locations
        start_loc = row['positions'][0]
        end_loc = row['positions'][1]
        
        start_x = start_loc['x'] * (120 / 100.0)
        start_y = start_loc['y'] * (80 / 100.0)
        end_x = end_loc['x'] * (120 / 100.0)
        end_y = end_loc['y'] * (80 / 100.0)
        
        # Calculate pass length and angle
        delta_x = end_x - start_x
        delta_y = end_y - start_y
        pass_length = math.sqrt(delta_x**2 + delta_y**2)
        pass_angle = math.atan2(delta_y, delta_x)
        
        return pd.Series([pass_length, pass_angle])
    else:
        return pd.Series([None, None])

In [27]:
passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


<ipython-input-27-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)
<ipython-input-27-53702553e163>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df[['calculated_pass_length', 'calculated_pass_angle']] = passes_df.apply(calculate_pass_details_from_single_column, axis=1)


In [28]:
def determine_pass_direction(angle):
    # Forward Pass: -π/4 < angle < π/4
    if -np.pi/4 < angle < np.pi/4:
        return "Forward Pass"
    # Backward Pass: 3π/4 < angle <= π or -π <= angle < -3π/4
    elif 3*np.pi/4 < angle <= np.pi or -np.pi <= angle < -3*np.pi/4:
        return "Backward Pass"
    # Side Pass: Otherwise
    else:
        return "Side Pass"

In [29]:
passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)

<ipython-input-29-a0b0c4f17a64>:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['pass_direction'] = passes_df['calculated_pass_angle'].apply(determine_pass_direction)


In [30]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1801}],14763,"[{'y': 50, 'x': 50}, {'y': 42, 'x': 38}]",2499719,Pass,1631,2H,2.836169,85,177960135,1631---2499719,2,15.758172,-2.723368,Backward Pass
1,8,High pass,[{'id': 1802}],8653,"[{'y': 42, 'x': 38}, {'y': 16, 'x': 74}]",2499719,Pass,1631,2H,5.311682,83,177960136,1631---2499719,2,47.946637,-0.448723,Forward Pass
2,8,Head pass,"[{'id': 1401}, {'id': 1801}]",167145,"[{'y': 84, 'x': 26}, {'y': 68, 'x': 32}]",2499719,Pass,1609,2H,8.449574,82,177960133,1609---2499719,1,14.686048,-1.058407,Side Pass
3,8,Simple pass,[{'id': 1802}],49876,"[{'y': 68, 'x': 32}, {'y': 100, 'x': 27}]",2499719,Pass,1609,2H,10.458703,85,177960134,1609---2499719,1,26.293725,1.801016,Side Pass
12,8,High pass,[{'id': 1802}],370224,"[{'y': 76, 'x': 8}, {'y': 75, 'x': 44}]",2499719,Pass,1609,2H,77.088925,83,177960140,1609---2499719,1,43.207407,-0.018516,Forward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498274,8,Simple pass,[{'id': 1801}],286994,"[{'y': 9, 'x': 28}, {'y': 16, 'x': 32}]",2565927,Pass,682,2H,886.983156,85,253301948,682---2565927,1,7.375636,0.862170,Side Pass
498276,8,Simple pass,[{'id': 1801}],20623,"[{'y': 41, 'x': 47}, {'y': 82, 'x': 60}]",2565927,Pass,682,2H,891.850784,85,253301950,682---2565927,2,36.320793,1.126851,Side Pass
498277,8,Simple pass,[{'id': 1801}],3529,"[{'y': 82, 'x': 60}, {'y': 66, 'x': 48}]",2565927,Pass,682,2H,895.998317,85,253301951,682---2565927,2,19.266551,-2.414950,Backward Pass
498278,8,Simple pass,[{'id': 1801}],364860,"[{'y': 66, 'x': 48}, {'y': 46, 'x': 52}]",2565927,Pass,682,2H,898.157657,85,253301953,682---2565927,2,16.704490,-1.279340,Side Pass


In [31]:
np.unique(passes_df.subEventName)

array(['Cross', 'Hand pass', 'Head pass', 'High pass', 'Launch',
       'Simple pass', 'Smart pass'], dtype=object)

In [32]:
passes_df

,eventId,subEventName,tags,playerId,positions,matchId,eventName,teamId,matchPeriod,eventSec,subEventId,id,uniqueTeamId,Zone,calculated_pass_length,calculated_pass_angle,pass_direction
0,8,Simple pass,[{'id': 1801}],14763,"[{'y': 50, 'x': 50}, {'y': 42, 'x': 38}]",2499719,Pass,1631,2H,2.836169,85,177960135,1631---2499719,2,15.758172,-2.723368,Backward Pass
1,8,High pass,[{'id': 1802}],8653,"[{'y': 42, 'x': 38}, {'y': 16, 'x': 74}]",2499719,Pass,1631,2H,5.311682,83,177960136,1631---2499719,2,47.946637,-0.448723,Forward Pass
2,8,Head pass,"[{'id': 1401}, {'id': 1801}]",167145,"[{'y': 84, 'x': 26}, {'y': 68, 'x': 32}]",2499719,Pass,1609,2H,8.449574,82,177960133,1609---2499719,1,14.686048,-1.058407,Side Pass
3,8,Simple pass,[{'id': 1802}],49876,"[{'y': 68, 'x': 32}, {'y': 100, 'x': 27}]",2499719,Pass,1609,2H,10.458703,85,177960134,1609---2499719,1,26.293725,1.801016,Side Pass
12,8,High pass,[{'id': 1802}],370224,"[{'y': 76, 'x': 8}, {'y': 75, 'x': 44}]",2499719,Pass,1609,2H,77.088925,83,177960140,1609---2499719,1,43.207407,-0.018516,Forward Pass
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
498274,8,Simple pass,[{'id': 1801}],286994,"[{'y': 9, 'x': 28}, {'y': 16, 'x': 32}]",2565927,Pass,682,2H,886.983156,85,253301948,682---2565927,1,7.375636,0.862170,Side Pass
498276,8,Simple pass,[{'id': 1801}],20623,"[{'y': 41, 'x': 47}, {'y': 82, 'x': 60}]",2565927,Pass,682,2H,891.850784,85,253301950,682---2565927,2,36.320793,1.126851,Side Pass
498277,8,Simple pass,[{'id': 1801}],3529,"[{'y': 82, 'x': 60}, {'y': 66, 'x': 48}]",2565927,Pass,682,2H,895.998317,85,253301951,682---2565927,2,19.266551,-2.414950,Backward Pass
498278,8,Simple pass,[{'id': 1801}],364860,"[{'y': 66, 'x': 48}, {'y': 46, 'x': 52}]",2565927,Pass,682,2H,898.157657,85,253301953,682---2565927,2,16.704490,-1.279340,Side Pass


In [33]:
#passes_count = passes_df.groupby('teamId')['subEventName'].value_counts().unstack(fill_value=0)

In [34]:
# Step 1: Combine 'pass_direction' and 'Zone' into a single column
passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)

# Step 2: Group by 'uniqueTeamId' and the new 'dir_zone' column, then count
direction_zone_count = passes_df.groupby(['uniqueTeamId', 'dir_zone']).size()

# Step 3: Pivot the results to have 'uniqueTeamId' as rows and each unique 'dir_zone' combination as columns
direction_zone_count_pivot = direction_zone_count.unstack(fill_value=0)

direction_count = passes_df.groupby('uniqueTeamId')['pass_direction'].value_counts().unstack(fill_value=0)

pass_direction_df = pd.concat([direction_zone_count_pivot, direction_count], axis=1)

<ipython-input-34-54e3dcb6a0f6>:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  passes_df['dir_zone'] = passes_df['pass_direction'].astype(str) + '_Zone_' + passes_df['Zone'].astype(str)


In [35]:
pass_direction_df

,Backward Pass_Zone_1,Backward Pass_Zone_2,Backward Pass_Zone_3,Forward Pass_Zone_1,Forward Pass_Zone_2,Forward Pass_Zone_3,Side Pass_Zone_1,Side Pass_Zone_2,Side Pass_Zone_3,Backward Pass,Forward Pass,Side Pass
uniqueTeamId,,,,,,,,,,,,
10531---2499726,3,4,0,14,10,3,13,14,5,7,27,32
10531---2499736,1,4,8,10,6,4,11,11,10,13,20,32
10531---2499741,1,2,0,5,7,1,4,6,1,3,13,11
10531---2499757,2,7,5,9,10,2,11,18,14,14,21,43
10531---2499766,3,3,0,13,2,0,6,2,0,6,15,8
...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0,4,2,4,7,2,7,17,7,6,13,31
756---2565892,0,5,3,6,11,3,6,29,12,8,20,47
756---2565904,2,8,5,11,6,1,6,14,3,15,18,23


## Pass Types

In [36]:
high_pass_types = ['High pass', 'Cross', 'Launch']

all_pass_types = passes_df['subEventName'].unique()
low_pass_types = [pt for pt in all_pass_types if pt not in high_pass_types and pt != 'Smart pass']

# --- overall ---
pass_counts = passes_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
pass_counts['Total_Passes'] = pass_counts.sum(axis=1)

# reindex to avoid missing columns (robust)
pass_counts['High_pass_Total'] = pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
pass_counts['Low_pass_Total']  = pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

pass_counts['High_pass_Ratio_Total'] = pass_counts['High_pass_Total'] / pass_counts['Total_Passes']
pass_counts['Low_pass_Ratio_Total']  = pass_counts['Low_pass_Total'] / pass_counts['Total_Passes']

final_pass_types_df = pass_counts[['Total_Passes', 'High_pass_Ratio_Total', 'Low_pass_Ratio_Total']].copy()

# --- zone-specific ---
zones = passes_df['Zone'].unique()
for zone in zones:
    zone_df = passes_df[passes_df['Zone'] == zone]

    zone_pass_counts = zone_df.groupby(['uniqueTeamId', 'subEventName']).size().unstack(fill_value=0)
    zone_pass_counts['Total_Passes'] = zone_pass_counts.sum(axis=1)

    zone_high_total = zone_pass_counts.reindex(columns=high_pass_types, fill_value=0).sum(axis=1)
    zone_low_total  = zone_pass_counts.reindex(columns=low_pass_types,  fill_value=0).sum(axis=1)

    # avoid divide-by-zero: teams with 0 passes in that zone -> NaN
    denom = zone_pass_counts['Total_Passes'].replace(0, pd.NA)

    final_pass_types_df[f'High_pass_Ratio_Zone{zone}'] = zone_high_total / denom
    final_pass_types_df[f'Low_pass_Ratio_Zone{zone}']  = zone_low_total / denom


In [37]:
final_pass_types_df

subEventName,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,Low_pass_Ratio_Zone1,High_pass_Ratio_Zone3,Low_pass_Ratio_Zone3
uniqueTeamId,,,,,,,,,
10531---2499726,66,0.166667,0.833333,0.107143,0.892857,0.200000,0.800000,0.250000,0.750000
10531---2499736,65,0.184615,0.800000,0.047619,0.952381,0.227273,0.772727,0.272727,0.681818
10531---2499741,27,0.259259,0.703704,0.200000,0.733333,0.300000,0.700000,0.500000,0.500000
10531---2499757,78,0.192308,0.782051,0.085714,0.914286,0.227273,0.772727,0.333333,0.571429
10531---2499766,29,0.241379,0.758621,0.000000,1.000000,0.318182,0.681818,NaN,NaN
...,...,...,...,...,...,...,...,...,...
756---2565880,50,0.320000,0.640000,0.214286,0.750000,0.272727,0.727273,0.636364,0.272727
756---2565892,75,0.146667,0.826667,0.111111,0.866667,0.083333,0.916667,0.277778,0.666667
756---2565904,56,0.250000,0.732143,0.107143,0.857143,0.421053,0.578947,0.333333,0.666667


# Pass_Connectivity

In [38]:
from collections import defaultdict
import networkx as nx

In [39]:
def calculate_pass_connectivity(group):
    # Initialize two empty graphs for the two teams in the match
    team_graphs = {team_id: nx.Graph() for team_id in group['uniqueTeamId'].unique()}
    
    # Track the previous player and team to identify consecutive passes
    prev_player = None
    prev_team = None
    
    for i, pass_ in group.iterrows():
        current_player = pass_['playerId']
        current_team = pass_['uniqueTeamId']
        
        # Check for consecutive passes within the same team
        if prev_team == current_team and prev_player is not None:
            team_graphs[current_team].add_edge(prev_player, current_player)
        
        # Update the previous player and team for the next iteration
        prev_player = current_player
        prev_team = current_team
    
    # Calculate connectivity for each team graph
    connectivities = {}
    for team_id, graph in team_graphs.items():
        if len(graph) == 0:  # Handle cases where a team made no passes or only incomplete passes
            connectivity = 0
        else:
            connectivity = nx.average_node_connectivity(graph)
        connectivities[team_id] = connectivity
    
    return connectivities

In [40]:
test_dfdf = passes_df.iloc[:1000]

In [41]:
connectivity_results = [calculate_pass_connectivity(group) for _, group in passes_df.groupby(['matchId'])]

In [42]:
connectivity_results

[{'1631---2499719': 1.8888888888888888, '1609---2499719': 4.444444444444445},
 {'1651---2499720': 1.9090909090909092, '1625---2499720': 4.6909090909090905},
 {'1610---2499721': 5.254545454545455, '1646---2499721': 1.8571428571428572},
 {'1673---2499722': 2.1515151515151514, '1628---2499722': 1.7636363636363637},
 {'1623---2499723': 2.7636363636363637, '1639---2499723': 2.6},
 {'1633---2499724': 4.218181818181818, '1611---2499724': 3.1944444444444446},
 {'1624---2499725': 4.136363636363637, '1613---2499725': 1.0},
 {'1619---2499726': 4.618181818181818, '10531---2499726': 3.1818181818181817},
 {'1612---2499727': 5.036363636363636, '1644---2499727': 2.690909090909091},
 {'1659---2499728': 4.090909090909091, '1627---2499728': 2.4909090909090907},
 {'1659---2499729': 3.709090909090909, '1644---2499729': 3.8363636363636364},
 {'1646---2499730': 4.533333333333333, '1627---2499730': 2.1272727272727274},
 {'1613---2499731': 2.4696969696969697, '1673---2499731': 4.072727272727272},
 {'1651---249

In [43]:
connectivity_df = [(k, v) for d in connectivity_results for k, v in d.items()]
connectivity_df = pd.DataFrame(connectivity_df, columns=['uniqueTeamId', 'Connectivity'])

In [44]:
connectivity_df.set_index('uniqueTeamId', inplace=True)
connectivity_df

,Connectivity
uniqueTeamId,
1631---2499719,1.888889
1609---2499719,4.444444
1651---2499720,1.909091
1625---2499720,4.690909
1610---2499721,5.254545
...,...
3315---2576336,3.581818
3204---2576337,2.242424
3164---2576337,3.755556


In [45]:
final_df = pd.concat([motif_ratios, final_pass_types_df, pass_direction_df, shot_df_final, connectivity_df], axis=1)
# Iterate through each column in final_df to find and adjust the specified columns
for col in final_df.columns:
    # Check if the column name contains 'Backward', 'Forward', or 'Side'
    if any(keyword in col for keyword in ['Backward', 'Forward', 'Side']):
        # Divide the column by the 'Total_Passes' column
        final_df[col] = final_df[col] / final_df['Total_Passes']
        # Rename the column to add " Ratio" at the end
        final_df.rename(columns={col: col + " Ratio"}, inplace=True)

In [46]:
final_df

,ABAB,ABCA,ABCB,ABCD,Total_Passes,High_pass_Ratio_Total,Low_pass_Ratio_Total,High_pass_Ratio_Zone2,Low_pass_Ratio_Zone2,High_pass_Ratio_Zone1,...,Side Pass_Zone_2 Ratio,Side Pass_Zone_3 Ratio,Backward Pass Ratio,Forward Pass Ratio,Side Pass Ratio,Total Shots,Far Shots Ratio,Middle Distance Shots Ratio,Near Shots Ratio,Connectivity
uniqueTeamId,,,,,,,,,,,,,,,,,,,,,
10531---2499726,0.031746,0.079365,0.063492,0.825397,66,0.166667,0.833333,0.107143,0.892857,0.200000,...,0.212121,0.075758,0.106061,0.409091,0.484848,3.0,0.333333,0.0,0.666667,3.181818
10531---2499736,0.032258,0.129032,0.145161,0.693548,65,0.184615,0.800000,0.047619,0.952381,0.227273,...,0.169231,0.153846,0.200000,0.307692,0.492308,1.0,1.000000,0.0,0.000000,3.145455
10531---2499741,0.041667,0.125000,0.083333,0.750000,27,0.259259,0.703704,0.200000,0.733333,0.300000,...,0.222222,0.037037,0.111111,0.481481,0.407407,1.0,0.000000,0.0,1.000000,1.250000
10531---2499757,0.040000,0.106667,0.106667,0.746667,78,0.192308,0.782051,0.085714,0.914286,0.227273,...,0.230769,0.179487,0.179487,0.269231,0.551282,2.0,0.000000,0.5,0.500000,3.963636
10531---2499766,0.076923,0.153846,0.192308,0.576923,29,0.241379,0.758621,0.000000,1.000000,0.318182,...,0.068966,0.000000,0.206897,0.517241,0.275862,NaN,NaN,NaN,NaN,1.022222
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
756---2565880,0.106383,0.085106,0.085106,0.723404,50,0.320000,0.640000,0.214286,0.750000,0.272727,...,0.340000,0.140000,0.120000,0.260000,0.620000,2.0,0.000000,0.5,0.500000,2.472727
756---2565892,0.041667,0.083333,0.069444,0.805556,75,0.146667,0.826667,0.111111,0.866667,0.083333,...,0.386667,0.160000,0.106667,0.266667,0.626667,2.0,0.500000,0.5,0.000000,3.818182
756---2565904,0.000000,0.075472,0.075472,0.849057,56,0.250000,0.732143,0.107143,0.857143,0.421053,...,0.250000,0.053571,0.267857,0.321429,0.410714,3.0,0.000000,1.0,0.000000,3.527273


In [47]:
table_name = "final_in_poss_df"

if half is not None:
    final_df.to_csv(f'{table_name}_{half}_{min_min}_{max_min}.csv', index=True)
else:
    final_df.to_csv(f'{table_name}.csv', index=True)

In [48]:
half

'2H'

# END OF FILE